In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## GPT 3.5 Judge

In [14]:
from openai import OpenAI

openai_api_key= os.getenv('OPENAI_API_KEY')
if openai_api_key is None:
    raise ValueError("API Key is not set.")

client = OpenAI(
    # This is the default and can be omitted
    api_key=openai_api_key,
)

def get_completion(prompt, model="gpt-3.5-turbo"):
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo",
    )
    return chat_completion.choices[0].message.content

## LLama 3

In [26]:
#https://github.com/ollama/ollama-python?tab=readme-ov-file
import ollama

def get_completion(prompt):
  response = ollama.chat(model='llama3', messages=[
    {
      'role': 'user',
      'content': prompt,
    },
  ])

  return response['message']['content']

In [27]:
def is_code_openai(row):
    text = row.str.cat(sep='\n')
    prompt = f"""
            I need you to analyze a question answer sequence of text and determine whether it contains code written 
            in any programming language. Please respond with "Yes" if the text contains code and 
            "No" if it does not. All langauages are valid, including markup languages.
            Example 1:
                Input: 
                    def hello_world():
                        print("Hello, world!")
                Output:
                    Yes
            Example 2:
                Input:
                    The quick brown fox jumps over the lazy dog.
                Output:
                    No

            It is important that you only answer with "Yes" or "No". No further explanation or justification
            is needed. You should be robust enough to handle code snippets in any programming language.
            Now, analyze the following text:
                Input:
                    {text}
                Output:         
            """
    answer = get_completion(prompt)
    return answer

## Main

In [28]:
from datasets import load_dataset
import pandas as pd

dataset_name = "yahma/alpaca-cleaned"
dataset = load_dataset(dataset_name)
df = pd.DataFrame(dataset['train'])
new_order = ['instruction', 'input', 'output']
df = df[new_order]
column_names = df.columns
     
for i, row in df.iterrows():
    print(is_code_openai(row))

    

No
No
No
No
No
